# OBA Live Data Exploration

In [1]:
# Bootstrap API KEY

import os
import requests

from dotenv import load_dotenv

# Get OBA API KEY
load_dotenv()


True

In [2]:
# CONSTS for API Calls

API_KEY = os.getenv("API_KEY")
API_ROOT_URL = "https://api.pugetsound.onebusaway.org/api/where"
ENDPOINT_TRIPS_FOR_ROUTE = "trips-for-route"

ROUTES = {
    48: 100228,
    8: 100275,
    62: 100252
}

"""
Returns a response object from the OBA API for the trips-for-route endpoint
REF: https://developer.onebusaway.org/api/where/methods/trips-for-route

include_schedule: boolean. Set this to `True` to include detailed
schedule information for the route.
"""
def query_trips_for_route(route_id, include_schedule=False):
    request_url = f'{API_ROOT_URL}/{ENDPOINT_TRIPS_FOR_ROUTE}/1_{ROUTES[route_id]}.json'
    response_data = requests.get(request_url, params={"key": API_KEY, "includeSchedule": include_schedule})
    return response_data.json()

## Make a map showing locations of vehicles associated with a given route

In [3]:
import folium

m = folium.Map(location=[47.617, -122.34], zoom_start=13)

trip_data = query_trips_for_route(8)

def generate_folium_marker_for_trip(trip_element):
    kw = {"prefix": "fa", "color": "green", "icon": "arrow-up"}
    angle = int(trip_element["status"]["orientation"])
    icon = folium.Icon(angle=angle, **kw)

    return folium.Marker(
        location=[trip_element["status"]["position"]["lat"], trip_element["status"]["position"]["lon"]],
        tooltip=trip_element["status"]["vehicleId"],
        popup=trip_element["status"]["phase"],
        icon=icon,
    )

# Generate markers for each trip,
# skipping trips that don't have a "status" key
markers = [generate_folium_marker_for_trip(trip) for trip in trip_data["data"]["list"] if "status" in trip.keys()]
[marker.add_to(m) for marker in markers]
# Display the map
m